# Step I - Import Libraries and get the list of all equity derivatives

In [27]:
import numpy as np
import pandas as pd
import datetime as dt
import pandas_datareader.data as pdr
import talib as ta
import math

In [28]:
stocks = pd.read_csv('NSE_derivatives_stocks.csv',encoding= 'unicode_escape')

In [29]:
ticker = stocks['ticker']

In [30]:
ticker = [item+'.NS' for item in ticker]

# Step II - Fetch data from Yahoo Finance API

In [31]:
start = dt.datetime.today() - dt.timedelta(360)
end = dt.datetime.today()

In [32]:
data = pdr.get_data_yahoo(ticker,start,end)

In [33]:
data_close = data['Adj Close']
price = data_close.copy()

In [34]:
price_pct = price.pct_change(1)

# Step III - Apply the scans

# 1. Price Change > 3%

In [35]:
scanned_stocks = []

In [36]:
for i in ticker:
    if price_pct[i][-1] > 0.03:
        scanned_stocks.append(i)

# 2. Price > 200 SMA and RSI(2) > 50

In [37]:
final_stocks = []

In [38]:
for i in scanned_stocks:
    if (ta.SMA(price[i],200)[-1] < price[i][-1]) & (ta.RSI(price[i],2)[-1] > 50):
        final_stocks.append(i)

In [39]:
cp_stocks = price[final_stocks].iloc[-1].round(1)

# Step IV - Find the Entry, Stop loss and Target price

In [40]:
entry_price = (cp_stocks*1.01).round(1)
entry_price = pd.DataFrame(entry_price)
sl_price = (entry_price* 1.03).round(1)
tgt_price = (entry_price* 0.94).round(1)

In [41]:
order_df = pd.concat([entry_price,sl_price,tgt_price],axis =1,ignore_index = True)

In [42]:
order_df.rename(columns = {0:'Entry_Price', 1:'Stop_Loss', 2:'Target'},inplace = True)

In [43]:
order_df

,Entry_Price,Stop_Loss,Target
Symbols,,,
CADILAHC.NS,340.3,350.5,319.9
NAUKRI.NS,2722.8,2804.5,2559.4


# Step V - Position Sizing

In [44]:
max_cap = 10000/order_df.shape[0]

In [45]:
order_df['Qty'] = np.floor(max_cap/order_df['Entry_Price'])

In [46]:
order_df['Max_Loss'] = order_df['Qty']*(order_df['Stop_Loss'] - order_df['Entry_Price'])

In [47]:
order_df['Max_Profit'] = order_df['Qty']*(order_df['Entry_Price'] - order_df['Target'])

In [48]:
order_df.loc['Total_Max_Loss','Max_Loss'] = order_df['Max_Loss'].sum()

In [49]:
order_df.loc['Total_Max_Profit','Max_Profit'] = order_df['Max_Profit'].sum()

In [50]:
order_df = order_df.replace(np.nan,'',regex = True)

In [51]:
order_df

,Entry_Price,Stop_Loss,Target,Qty,Max_Loss,Max_Profit
Symbols,,,,,,
CADILAHC.NS,340.3,350.5,319.9,14,142.8,285.6
NAUKRI.NS,2722.8,2804.5,2559.4,1,81.7,163.4
Total_Max_Loss,,,,,224.5,
Total_Max_Profit,,,,,,449


In [52]:
order_df.to_csv('trade-data/2020-05-15.csv')